In [1]:
%cd /content/drive/MyDrive/PRPipe

/content/drive/MyDrive/PRPipe


In [2]:
import pandas as pd
import json
import urllib
from tqdm import tqdm

# **Load Data**

In [35]:
f = open('./ReNewEfArgRanksFinalPN-Graph.json',)

In [36]:
data = json.load(f)

In [37]:
f.close()

In [38]:
nodes = []
for i in tqdm(data['Graph']):
    px = []
    px.append(int(i['id']))
    px.append(i['label'])
    px.append(i['concept'])
    px.append(i['type'])
    px.append(i['ranks'])
    px.append(i['final_ranks'])
    px.append(i['Pos'])
    px.append(i['Neg'])
    px.append(i['pRank'])
    px.append(i['nRank'])
    nodes.append(px)
  
# Closing file
f.close()

100%|██████████| 337682/337682 [00:01<00:00, 266435.49it/s]


In [39]:
rels = []
for i in tqdm(data['Relationship']):
  px = []
  px.append(int(i['start']))
  px.append(int(i['end']))
  px.append(i['effect'])
  px.append(i['type'])
  rels.append(px)
  
# Closing file
f.close()

100%|██████████| 2033004/2033004 [00:04<00:00, 465993.74it/s]


In [40]:
xd = pd.DataFrame(
            nodes, columns=['id', 'label', 'concept', 'type', 'ranks', 'final_ranks', 'Pos', 'Neg', 'pRank', 'nRank'])
xr = pd.DataFrame(
            rels, columns=['start', 'end', 'effect', 'type'])

In [41]:
xd.head()

,id,label,concept,type,ranks,final_ranks,Pos,Neg,pRank,nRank
0,0,accident,accident,node,17399.134735,1552.897128,0.000000,17508.337566,4.442049e-07,1584.633381
1,1,death,death,node,23440.786330,11702.120120,13.324650,23751.858104,4.442049e-07,11357.284706
2,2,disease,disease,node,26235.653549,5132.346620,2072.677657,24777.582845,7.311114e+02,5160.155460
3,3,pneumonium,pneumonium,node,6641.914576,1967.091368,273.826409,6487.896418,9.672191e+02,1484.111504
4,4,cancer,cancer,node,20368.378862,3051.568576,0.000000,20963.631060,4.442049e-07,3088.935625


In [42]:
xr.head()

,start,end,effect,type
0,0,1,negative,relationship
1,0,10,negative,relationship
2,0,89,negative,relationship
3,0,53,negative,relationship
4,0,55,negative,relationship


# **Load Input Data**

In [11]:
df = pd.read_excel('Evaluation_v41.xlsx')
df.head()

,Input claim,Target,graph target
0,Modern technology is a disadvantage to society,Modern technology,modern technology
1,Airplane passengers should pay for their bodyw...,Airplane passengers,aircraft in flight
2,We should adapt to climate change rather than ...,to climate change,climate change
3,Should students be allowed to skip grades?,allowed to skip grades,skip grades
4,Is Artificial General Intelligence (AGI) a thr...,Artificial General Intelligence ( AGI ),artificial intelligence


In [12]:
g = df['Target'].tolist()

In [13]:
g[:10]

['Modern technology',
 'Airplane passengers',
 'to climate change',
 'allowed to skip grades',
 'Artificial General Intelligence ( AGI )',
 'School be Mandatory',
 'Social Media',
 'Religion',
 'allowed to vote on abortion legislation',
 'Feminism']

# **Fast Fiass**

In [15]:
!pip install faiss-gpu
!pip install requests
!pip install sentence_transformers
!pip install --user -U nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85.5 MB 113 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.9 MB/s 
     |████████████████████████████████| 5.3 MB 60.5 MB/s 
     |████████████████████████████████| 1.3 MB 45.7 MB/s 
     |████████████████████████████████| 163 kB 59.7 MB/s 
     |████████████████████████████████| 7.6 MB 67.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=931463379e8a054369127b6f47b2cda99b3837a8e832ab237549fd9057e63fd2
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
Looking in indexes: https://

In [16]:
import numpy as np
import faiss
import requests
from io import StringIO
import pandas as pd
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from datetime import datetime
from sentence_transformers import SentenceTransformer
import os
import nltk
nltk.download("punkt")
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
datax = xd
datax.head()

,id,label,concept,type,ranks,final_ranks,Pos,Neg,pRank,nRank
0,0,accident,accident,node,17399.134735,1552.897128,0.000000,17508.337566,4.442049e-07,1584.633381
1,1,death,death,node,23440.786330,11702.120120,13.324650,23751.858104,4.442049e-07,11357.284706
2,2,disease,disease,node,26235.653549,5132.346620,2072.677657,24777.582845,7.311114e+02,5160.155460
3,3,pneumonium,pneumonium,node,6641.914576,1967.091368,273.826409,6487.896418,9.672191e+02,1484.111504
4,4,cancer,cancer,node,20368.378862,3051.568576,0.000000,20963.631060,4.442049e-07,3088.935625


In [18]:
sentences = datax['concept'].tolist()
sentences[:10]

['accident',
 'death',
 'disease',
 'pneumonium',
 'cancer',
 'heart attack',
 'illnes',
 'stroke',
 'complication',
 'infection']

In [19]:
sentences_2 = datax['concept'].tolist()
sentences_2[:10]

['accident',
 'death',
 'disease',
 'pneumonium',
 'cancer',
 'heart attack',
 'illnes',
 'stroke',
 'complication',
 'infection']

In [20]:
sentences_id = datax['id'].tolist()
sentences_id[:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [21]:
sentences = [
    sentence.replace('\n', '') for sentence in list(sentences) if type(sentence) is str
    ]

In [22]:
with open('backup_sentences.txt', 'w') as fp:
    fp.write('\n'.join(sentences))

In [24]:
import torch

In [25]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [26]:
model = SentenceTransformer('bert-base-nli-mean-tokens', device='cuda').to(device)

sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

(337682, 768)

In [27]:
sentence_embeddings.shape[0]

337682

In [28]:
directory = "sim_sentences"
parent_dir = "./"
path = os.path.join(parent_dir, directory) 
try: 
    os.mkdir(path) 
except OSError as error: 
    print(error)  

In [29]:
with open(f'./sim_sentences/embeddings_X.npy', 'wb') as fp:
    np.save(fp, sentence_embeddings[0:256])

In [30]:
split = 256
file_count = 0
for i in range(0, sentence_embeddings.shape[0], split):
    end = i + split
    if end > sentence_embeddings.shape[0] + 1:
        end = sentence_embeddings.shape[0] + 1
    file_count = '0' + str(file_count) if file_count < 0 else str(file_count)
    with open(f'./sim_sentences/embeddings_{file_count}.npy', 'wb') as fp:
        np.save(fp, sentence_embeddings[i:end, :])
    print(f"embeddings_{file_count}.npy | {i} -> {end}")
    file_count = int(file_count) + 1

embeddings_0.npy | 0 -> 256
embeddings_1.npy | 256 -> 512
embeddings_2.npy | 512 -> 768
embeddings_3.npy | 768 -> 1024
embeddings_4.npy | 1024 -> 1280
embeddings_5.npy | 1280 -> 1536
embeddings_6.npy | 1536 -> 1792
embeddings_7.npy | 1792 -> 2048
embeddings_8.npy | 2048 -> 2304
embeddings_9.npy | 2304 -> 2560
embeddings_10.npy | 2560 -> 2816
embeddings_11.npy | 2816 -> 3072
embeddings_12.npy | 3072 -> 3328
embeddings_13.npy | 3328 -> 3584
embeddings_14.npy | 3584 -> 3840
embeddings_15.npy | 3840 -> 4096
embeddings_16.npy | 4096 -> 4352
embeddings_17.npy | 4352 -> 4608
embeddings_18.npy | 4608 -> 4864
embeddings_19.npy | 4864 -> 5120
embeddings_20.npy | 5120 -> 5376
embeddings_21.npy | 5376 -> 5632
embeddings_22.npy | 5632 -> 5888
embeddings_23.npy | 5888 -> 6144
embeddings_24.npy | 6144 -> 6400
embeddings_25.npy | 6400 -> 6656
embeddings_26.npy | 6656 -> 6912
embeddings_27.npy | 6912 -> 7168
embeddings_28.npy | 7168 -> 7424
embeddings_29.npy | 7424 -> 7680
embeddings_30.npy | 7680 -> 7

In [31]:
d = sentence_embeddings.shape[1]
d

768

In [32]:
nlist = 50
quantizer = faiss.IndexFlatL2(d)

In [33]:
res = faiss.StandardGpuResources()

In [34]:
f_index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)

In [35]:
index = faiss.index_cpu_to_gpu(res, 0, f_index)

In [36]:
index.is_trained

False

In [37]:
assert not index.is_trained

In [38]:
index.train(sentence_embeddings)
index.is_trained

True

In [39]:
index.add(sentence_embeddings)
index.ntotal

337682

In [40]:
k = 1
xq = model.encode(["O Brasil ser/u00e1 beneficiado com a privatiza"])
index.nprobe = 10

In [41]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[74356]]
CPU times: user 9.54 ms, sys: 990 µs, total: 10.5 ms
Wall time: 12.2 ms


In [42]:
[f'{i}: {sentences[i]}' for i in I[0]]

['74356: O Brasil ser u00e1 beneficiado com a privatiza u00e7 u00e3o das administra u00e7 u00f5es de seus principais portos?']

In [43]:
mapp = I[0][0]
print(I[0][0])

74356


In [44]:
sentences[I[0][0]]

'O Brasil ser u00e1 beneficiado com a privatiza u00e7 u00e3o das administra u00e7 u00f5es de seus principais portos?'

# **Preparing Inputs**

In [21]:
datay = xr

In [22]:
stdf = datay.groupby(['start'])

In [ ]:
!pip install pyenchant
!apt install enchant --fix-missing

In [24]:
import enchant
import string
dnc = enchant.Dict("en_US")

In [25]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [26]:
def isfloat(num):
    try:
        float(num)
        return True
    except ValueError:
        return False

In [27]:
stdf = datay.groupby(['start'])

In [28]:
def checkIfSubject(f):
  bool = False
  id = datax.id[datax.label == f].tolist()[0]
  #print(id)
  if id in stdf.groups.keys():
    bool = True
  return bool

In [29]:
def perfectmatch(sent):
  result = True
  flag = 0
  text_tokens = word_tokenize(sent)
  if len(text_tokens) > 3:
    result = False
  elif text_tokens == 'nan':
    result = False
  else:
    for x in text_tokens:
      if dnc.check(str(x)) == False:
        flag = 1
      if flag == 1:
        result = False
      if type(x) != str:
        result = False
      if isfloat(x):
        result = False
  return result

In [54]:
def fiass_S(strn):
  k = 500
  xq = model.encode([strn])
  D, I = index.search(xq, k)
  X = ""
  for i in range(500):
    if perfectmatch(sentences[I[0][i]]):
      if checkIfSubject(sentences[I[0][i]]):
        X = str(sentences[I[0][i]])
        if X != "":
          break
  return X

In [55]:
ids = []
tps = []
for i in g:
  xi = i
  gtar = fiass_S(xi)
  tps.append(gtar)
  idy = xd.id[xd.label == gtar].tolist()[0]
  ids.append(idy)

In [56]:
ids

[49142,
 37472,
 61,
 75535,
 16465,
 68965,
 74658,
 74660,
 337388,
 74388,
 74388,
 75120,
 74851,
 54705,
 77284,
 35494,
 6777,
 10352,
 642,
 75068,
 74369,
 6799,
 76280,
 76463,
 66092,
 76206,
 74689,
 45052,
 35572,
 9591,
 773,
 12964,
 30130,
 76085,
 29916,
 75816,
 75876,
 75101,
 74388,
 74883]

In [ ]:
df['graph target'] = tps
df['target id'] = ids

In [44]:
df.head()

,Unnamed: 0,Input claim,Target,graph target,target id
0,0,Modern technology is a disadvantage to society,Modern technology,modern technology,49142
1,1,Airplane passengers should pay for their bodyw...,Airplane passengers,aircraft in flight,37472
2,2,We should adapt to climate change rather than ...,to climate change,climate change,61
3,3,Should students be allowed to skip grades?,allowed to skip grades,to skip grades,75535
4,4,Is Artificial General Intelligence (AGI) a thr...,Artificial General Intelligence ( AGI ),mechanization,16465


In [59]:
df.to_csv('rev_v42.csv')

# **Aspect extractor**

In [45]:
stdf = xr.groupby(['start'])

In [46]:
posr = xd['pRank'].tolist()
negr = xd['nRank'].tolist()

In [47]:
posr[121542]

1.1653528127602848

In [48]:
def get_pro(id, param):
  tid = 0
  fsen = ""
  count = 0
  emoScore = 0
  biggest = ''
  ndf = stdf.get_group(int(id))
  start = ndf['start'].tolist()
  enc = ndf['end'].tolist()
  for x in range(len(start)):
    impact = 0
    impact = posr[enc[x]]
    count += 1
    if impact > param:
      param = impact
      tid = enc[x]
      s = xd.label[xd.id == tid].tolist()[0]
      biggest = s
      if type(s) == str:
        if perfectmatch(s):
          fsen = s
  if fsen == "":
    fsen = biggest
  #print(count)
  return fsen

In [49]:
def get_con(id, param):
  tid = 0
  fsen = ""
  count = 0
  emoScore = 0
  biggest = ''
  ndf = stdf.get_group(int(id))
  start = ndf['start'].tolist()
  enc = ndf['end'].tolist()
  for x in range(len(start)):
    impact = 0
    impact = negr[enc[x]]
    count += 1
    if impact > param:
      param = impact
      tid = enc[x]
      s = xd.label[xd.id == tid].tolist()[0]
      biggest = s
      if type(s) == str:
        if perfectmatch(s):
          fsen = s
  if fsen == "":
    fsen = biggest
  #print(count)
  return fsen

In [17]:
def pAss(p, z, t, i):
  emo = 'CON'
  oxi = z
  oxp = t
  id = i
  type = emo
  if emo == "PRO":
    print("Finding PRO")
    fin = get_pro(id, 0)
    return oxi, oxp, type, fin
  else:
    print("Finding CON")
    fin = get_con(id, 0)
    return oxi, oxp, type, fin

# **Initialize Pipe**

In [11]:
df = pd.read_csv('rev_v42.csv')
df.head()

,Unnamed: 0,Input claim,Target,graph target,target id
0,0,Modern technology is a disadvantage to society,Modern technology,modern technology,49142
1,1,Airplane passengers should pay for their bodyw...,Airplane passengers,aircraft in flight,37472
2,2,We should adapt to climate change rather than ...,to climate change,climate change,61
3,3,Should students be allowed to skip grades?,allowed to skip grades,to skip grades,75535
4,4,Is Artificial General Intelligence (AGI) a thr...,Artificial General Intelligence ( AGI ),mechanization,16465


In [18]:
prompt = df['Input claim'].tolist()
tars = df['Target'].tolist()
graph = df['graph target'].tolist()
tid_n = df['target id'].tolist()

In [50]:
data = []
for x in tqdm(range(len(prompt))):
  gz = []
  B, X, Y, Z = pAss(prompt[x], tars[x], graph[x], tid_n[x])
  X = str(X)
  Y = str(Y)
  Z = str(Z)
  XY = X,Y,Z
  finalcut = ' '.join(XY)
  gz.append(prompt[x])
  gz.append(B)
  gz.append(finalcut)
  gz.append('')
  gz.append('')
  data.append(gz)
  print(f'Target: {B} \t Input: {finalcut}')

  0%|          | 0/40 [00:00<?, ?it/s]

Finding CON


100%|██████████| 40/40 [00:00<00:00, 62.47it/s]

Target: Modern technology 	 Input: modern technology CON environmental health hazard
Finding CON
Target: Airplane passengers 	 Input: aircraft in flight CON damage
Finding CON
Target: to climate change 	 Input: climate change CON the death
Finding CON
Target: allowed to skip grades 	 Input: to skip grades CON Their age with not
Finding CON
Target: Artificial General Intelligence ( AGI ) 	 Input: mechanization CON industrial revolution
Finding CON
Target: School be Mandatory 	 Input: school choice CON 2018 teacher strike in the united state
Finding CON
Target: Social Media 	 Input: Social Media CON   and bodies
Finding CON
Target: Religion 	 Input: Religion CON   blind
Finding CON
Target: allowed to vote on abortion legislation 	 Input: law allowing abortion CON avoid considering an embryo to be
Finding CON
Target: Feminism 	 Input: Feminism CON movement
Finding CON
Target: Feminism 	 Input: Feminism CON movement
Finding CON
Target: animal testing banned 	 Input: Ban Communist Symbols C

In [52]:
xd = pd.DataFrame(
            data, columns=['Input claim', 'Target', 'Aspects from the knowledge graph', 'Arguments generated from the PPLM model', 'Final output'])

name = './Evaluation_v43.xlsx'
writer = pd.ExcelWriter(name)
xd.to_excel(writer)
writer.save()